# FinBERT Profiling: Medium FinBert + Knowledge Distilation

This notebook is intentionally **thin**: it reuses the profiling utilities in `pipelines/finBERT/finbert/` (especially `finbert/finbert_profile.py` and `finbert/profile_utils.py`) instead of copying large code blocks.




In [81]:
from __future__ import annotations

from pathlib import Path
import shutil
import os
import logging
import sys
sys.path.append('..')

from textblob import TextBlob
from pprint import pprint
from sklearn.metrics import classification_report

from transformers import AutoModelForSequenceClassification

from finbert.finbert import *
from finbert.finbert_profile import *
from finbert.profile_utils import get_model_size_mb, print_device_info, setup_nltk_data
import finbert.utils as tools

from finbert.finbert_kd_trainer import KDFinBert



%load_ext autoreload
%autoreload 2

project_dir = Path.cwd().parent
pd.set_option('max_colwidth', None)

import wandb

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [82]:
wandb.init(
    entity="si2449-columbia-university",
    project="finbert-experiments",
    name="training-distilled_bert-profiled",
    group="knowledge-distillation",
)

In [83]:
cl_path = project_dir/'models'/'student'
cl_data_path = project_dir/'data'/'sentiment_data'

In [84]:
try:
    shutil.rmtree(cl_path) 
except:
    pass

bertmodel = AutoModelForSequenceClassification.from_pretrained("prajjwal1/bert-medium", cache_dir=None, num_labels=3)


config = Config(   data_dir=cl_data_path,
                   bert_model=bertmodel,
                   num_train_epochs=4,
                   model_dir=cl_path,
                   max_seq_length = 48,
                   train_batch_size = 32,
                   learning_rate = 1e-5,
                   output_mode='classification',
                   warm_up_proportion=0.1,
                   local_rank=-1,
                   discriminate=True,
                   gradual_unfreeze=True,
                   encoder_no = 8)

config.profile_train_steps = 20
config.num_hidden_layers = 8

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at prajjwal1/bert-medium and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [85]:
from contextlib import nullcontext
from typing import Any

import torch.nn.functional as F
from sklearn.metrics import f1_score


def timed_eval(
    *, finbert: FinBert, model: torch.nn.Module, examples, use_amp: bool
) -> tuple[pd.DataFrame, dict[str, Any]]:
    """Evaluation loop with optional CUDA autocast + timing (kept small for notebook use)."""
    loader = finbert.get_loader(examples, phase="eval")
    device = finbert.device

    model.eval()
    preds: list[np.ndarray] = []
    labels: list[int] = []

    if device.type == "cuda":
        torch.cuda.synchronize(device)

    start = time.perf_counter()
    with torch.no_grad():
        for batch in loader:
            batch = tuple(t.to(device) for t in batch)
            input_ids, attention_mask, token_type_ids, label_ids, _agree_ids = batch

            logits = model(input_ids=input_ids, attention_mask=attention_mask)[0]

            preds.extend(logits.detach().cpu().numpy())
            labels.extend(label_ids.detach().cpu().numpy().tolist())

    if device.type == "cuda":
        torch.cuda.synchronize(device)

    wall_s = time.perf_counter() - start
    n = len(labels)

    results_df = pd.DataFrame({"predictions": preds, "labels": labels})

    timing = {
        "eval_wall_s": float(wall_s),
        "eval_num_samples": int(n),
        "eval_samples_per_s": float(n / wall_s) if wall_s > 0 else float("inf"),
    }
    return results_df, timing

In [86]:
finbert = ProfiledFinBert(config)
finbert.config.base_model = 'prajjwal1/bert-medium'
finbert.config.discriminate=True
finbert.config.gradual_unfreeze=True

In [87]:
finbert.prepare_model(label_list=['positive','negative','neutral'])

12/16/2025 16:36:57 - INFO - finbert.finbert -   device: cuda n_gpu: 1, distributed training: False, 16-bits training: False


In [88]:
train_data = finbert.get_data('train')

In [89]:
model = finbert.create_the_model()

In [91]:
start = time.perf_counter()
trained_model = finbert.train(train_examples = train_data, model = model)
train_wall_s = time.perf_counter() - start

12/16/2025 16:36:59 - INFO - finbert.utils -   *** Example ***
12/16/2025 16:36:59 - INFO - finbert.utils -   guid: train-1
12/16/2025 16:36:59 - INFO - finbert.utils -   tokens: [CLS] after the reporting period , bio ##tie north american licensing partner so ##max ##on pharmaceuticals announced positive results with na ##lm ##efe ##ne in a pilot phase 2 clinical trial for smoking ce ##ssa ##tion [SEP]
12/16/2025 16:36:59 - INFO - finbert.utils -   input_ids: 101 2044 1996 7316 2558 1010 16012 9515 2167 2137 13202 4256 2061 17848 2239 24797 2623 3893 3463 2007 6583 13728 27235 2638 1999 1037 4405 4403 1016 6612 3979 2005 9422 8292 11488 3508 102 0 0 0 0 0 0 0 0 0 0 0
12/16/2025 16:36:59 - INFO - finbert.utils -   attention_mask: 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0
12/16/2025 16:36:59 - INFO - finbert.utils -   token_type_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
12/16/

12/16/2025 16:37:00 - INFO - finbert.finbert -   ***** Loading data *****
12/16/2025 16:37:00 - INFO - finbert.finbert -     Num examples = 3488
12/16/2025 16:37:00 - INFO - finbert.finbert -     Batch size = 32
12/16/2025 16:37:00 - INFO - finbert.finbert -     Num steps = 48



Starting Profiled Training
Device: cuda
Profiling activities: [<ProfilerActivity.CPU: 0>, <ProfilerActivity.CUDA: 2>]



Epoch:   0%|          | 0/4 [00:00<?, ?it/s]



Profiling complete for first epoch (20 steps)
Continuing full training without profiling...


PROFILING RESULTS - Training


By CPU Time:
-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                                   Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg     Self CUDA   Self CUDA %    CUDA total  CUDA time avg       CPU Mem  Self CPU Mem      CUDA Mem  Self CUDA Mem    # of Calls  
-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                               aten::to         0.11%     722.450us        52.75%

...

Iteration: 100%|██████████| 109/109 [00:08<00:00, 12.52it/s]
12/16/2025 16:37:30 - INFO - finbert.utils -   *** Example ***
12/16/2025 16:37:30 - INFO - finbert.utils -   guid: validation-1
12/16/2025 16:37:30 - INFO - finbert.utils -   tokens: [CLS] our in - depth expertise extends to the fields of energy , industry , urban & mobility and water & environment [SEP]
12/16/2025 16:37:30 - INFO - finbert.utils -   input_ids: 101 2256 1999 1011 5995 11532 8908 2000 1996 4249 1997 2943 1010 3068 1010 3923 1004 12969 1998 2300 1004 4044 102 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
12/16/2025 16:37:30 - INFO - finbert.utils -   attention_mask: 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
12/16/2025 16:37:30 - INFO - finbert.utils -   token_type_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
12/16/2025 16:37:30 - INFO - finbert.utils -   label: neutral (id = 2)
12/16/2025 16:37:30 

Validation losses: [1.072305679321289, 0.9563326927331778, 0.8792280508921697]


Iteration: 100%|██████████| 109/109 [00:09<00:00, 12.10it/s]
12/16/2025 16:37:39 - INFO - finbert.utils -   *** Example ***
12/16/2025 16:37:39 - INFO - finbert.utils -   guid: validation-1
12/16/2025 16:37:39 - INFO - finbert.utils -   tokens: [CLS] our in - depth expertise extends to the fields of energy , industry , urban & mobility and water & environment [SEP]
12/16/2025 16:37:39 - INFO - finbert.utils -   input_ids: 101 2256 1999 1011 5995 11532 8908 2000 1996 4249 1997 2943 1010 3068 1010 3923 1004 12969 1998 2300 1004 4044 102 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
12/16/2025 16:37:39 - INFO - finbert.utils -   attention_mask: 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
12/16/2025 16:37:39 - INFO - finbert.utils -   token_type_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
12/16/2025 16:37:39 - INFO - finbert.utils -   label: neutral (id = 2)
12/16/2025 16:37:39 

Validation losses: [1.072305679321289, 0.9563326927331778, 0.8792280508921697, 0.8541812254832342]


Epoch: 100%|██████████| 4/4 [00:31<00:00,  7.85s/it]


In [ ]:
test_data = finbert.get_data("test")

results = finbert.evaluate(examples=test_data, model=trained_model)

eval_df, eval_timing = timed_eval(
    finbert=finbert, model=trained_model, examples=test_data, use_amp=False
)

12/16/2025 16:37:41 - INFO - finbert.utils -   *** Example ***
12/16/2025 16:37:41 - INFO - finbert.utils -   guid: test-1
12/16/2025 16:37:41 - INFO - finbert.utils -   tokens: [CLS] the bristol port company has sealed a one million pound contract with cooper specialised handling to supply it with four 45 - ton ##ne , custom ##ised reach stack ##ers from ko ##ne ##cr ##ane ##s [SEP]
12/16/2025 16:37:41 - INFO - finbert.utils -   input_ids: 101 1996 7067 3417 2194 2038 10203 1037 2028 2454 9044 3206 2007 6201 17009 8304 2000 4425 2009 2007 2176 3429 1011 10228 2638 1010 7661 5084 3362 9991 2545 2013 12849 2638 26775 7231 2015 102 0 0 0 0 0 0 0 0 0 0
12/16/2025 16:37:41 - INFO - finbert.utils -   attention_mask: 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0
12/16/2025 16:37:41 - INFO - finbert.utils -   token_type_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
12/16/2025 16:37:41 - IN

12/16/2025 16:37:41 - INFO - finbert.finbert -     Num steps = 120
12/16/2025 16:37:41 - INFO - finbert.finbert -   ***** Running evaluation ***** 
12/16/2025 16:37:41 - INFO - finbert.finbert -     Num examples = 970
12/16/2025 16:37:41 - INFO - finbert.finbert -     Batch size = 32
Testing: 100%|██████████| 31/31 [00:00<00:00, 38.20it/s]


In [93]:
def report(df, cols=['label','prediction','logits']):
    #print('Validation loss:{0:.2f}'.format(metrics['best_validation_loss']))
    cs = CrossEntropyLoss(weight=finbert.class_weights)
    loss = cs(torch.tensor(list(df[cols[2]])),torch.tensor(list(df[cols[0]])))
    print("Loss:{0:.2f}".format(loss))
    print("Accuracy:{0:.2f}".format((df[cols[0]] == df[cols[1]]).sum() / df.shape[0]) )
    print("\nClassification Report:")
    return_val = classification_report(df[cols[0]], df[cols[1]], output_dict=True)
    print(return_val)
    return_val["Loss"] = loss
    return_val["Accuracy"] = (df[cols[0]] == df[cols[1]]).sum() / df.shape[0]
    return return_val

In [94]:
results['prediction'] = results.predictions.apply(lambda x: np.argmax(x,axis=0))

In [ ]:
wandb_report = report(results,cols=['labels','prediction','predictions'])


Loss:0.88
Accuracy:0.64

Classification Report:
{'0': {'precision': 0.4530386740331492, 'recall': 0.30711610486891383, 'f1-score': 0.36607142857142855, 'support': 267.0}, '1': {'precision': 0.3904761904761905, 'recall': 0.640625, 'f1-score': 0.48520710059171596, 'support': 128.0}, '2': {'precision': 0.7944732297063903, 'recall': 0.8, 'f1-score': 0.7972270363951474, 'support': 575.0}, 'accuracy': 0.643298969072165, 'macro avg': {'precision': 0.5459960314052433, 'recall': 0.582580368289638, 'f1-score': 0.5495018551860973, 'support': 970.0}, 'weighted avg': {'precision': 0.6471797787927605, 'recall': 0.643298969072165, 'f1-score': 0.6373743569397122, 'support': 970.0}}


In [96]:
teacher_path = project_dir/'models'/'teacher'
teacher = AutoModelForSequenceClassification.from_pretrained(
    teacher_path, num_labels=3, cache_dir=None
)
student = trained_model

new_path = project_dir/'models'/'distilled_student'

In [ ]:
config = Config(   data_dir=cl_data_path,
                   bert_model=None,
                   num_train_epochs=2,
                   model_dir=new_path,
                   max_seq_length = 48,
                   train_batch_size = 32,
                   learning_rate = 5e-6,
                   output_mode='classification',
                   warm_up_proportion=0.1,
                   local_rank=-1,
                   discriminate=True,
                   gradual_unfreeze=True,
                   encoder_no = 4,
                   base_model='prajjwal1/bert-medium')


In [98]:
kd = KDFinBert(teacher=teacher, student=student, config=config)

In [99]:
kd.prepare_model(label_list=["positive", "negative", "neutral"])

In [100]:
train_data = kd.get_data('train')
model = kd.create_the_model()

In [101]:
start = time.perf_counter()
trained_model = kd.train(train_examples = train_data, teacher=teacher, student=student)
train_wall_s = time.perf_counter() - start

12/16/2025 16:37:46 - INFO - finbert.utils -   *** Example ***
12/16/2025 16:37:46 - INFO - finbert.utils -   guid: train-1
12/16/2025 16:37:46 - INFO - finbert.utils -   tokens: [CLS] after the reporting period , bio ##tie north american licensing partner so ##max ##on pharmaceuticals announced positive results with na ##lm ##efe ##ne in a pilot phase 2 clinical trial for smoking ce ##ssa ##tion [SEP]
12/16/2025 16:37:46 - INFO - finbert.utils -   input_ids: 101 2044 1996 7316 2558 1010 16012 9515 2167 2137 13202 4256 2061 17848 2239 24797 2623 3893 3463 2007 6583 13728 27235 2638 1999 1037 4405 4403 1016 6612 3979 2005 9422 8292 11488 3508 102 0 0 0 0 0 0 0 0 0 0 0
12/16/2025 16:37:46 - INFO - finbert.utils -   attention_mask: 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0
12/16/2025 16:37:46 - INFO - finbert.utils -   token_type_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
12/16/

12/16/2025 16:37:46 - INFO - finbert.finbert_kd_trainer -   ***** Loading data *****
12/16/2025 16:37:46 - INFO - finbert.finbert_kd_trainer -     Num examples = 3488
12/16/2025 16:37:46 - INFO - finbert.finbert_kd_trainer -     Batch size = 32
12/16/2025 16:37:46 - INFO - finbert.finbert_kd_trainer -     Num steps = 24
/home/srm2245/hpml-project/pipelines/finBERT/notebooks/../finbert/finbert_kd_trainer.py:314: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler(enabled=self.config.use_amp)
Epoch:   0%|          | 0/2 [00:00<?, ?it/s]/home/srm2245/hpml-project/pipelines/finBERT/notebooks/../finbert/finbert_kd_trainer.py:356: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=self.config.use_amp):
Epoch:   0%|          | 0/2 [00:00<?, ?it/s]/home/srm2245/hpml-project/

Validation losses: [0.7298611815159137]


Iteration: 100%|██████████| 109/109 [00:18<00:00,  5.76it/s]
12/16/2025 16:38:46 - INFO - finbert.utils -   *** Example ***
12/16/2025 16:38:46 - INFO - finbert.utils -   guid: validation-1
12/16/2025 16:38:46 - INFO - finbert.utils -   tokens: [CLS] our in - depth expertise extends to the fields of energy , industry , urban & mobility and water & environment [SEP]
12/16/2025 16:38:46 - INFO - finbert.utils -   input_ids: 101 2256 1999 1011 5995 11532 8908 2000 1996 4249 1997 2943 1010 3068 1010 3923 1004 12969 1998 2300 1004 4044 102 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
12/16/2025 16:38:46 - INFO - finbert.utils -   attention_mask: 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
12/16/2025 16:38:46 - INFO - finbert.utils -   token_type_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
12/16/2025 16:38:46 - INFO - finbert.utils -   label: neutral (id = 2)
12/16/2025 16:38:46 

Validation losses: [0.7298611815159137, 0.6765608145640447]


Epoch: 100%|██████████| 2/2 [00:38<00:00, 19.15s/it]


In [102]:
test_data = kd.get_data("test")

results = kd.evaluate(examples=test_data, model=trained_model)



12/16/2025 16:38:48 - INFO - finbert.utils -   *** Example ***
12/16/2025 16:38:48 - INFO - finbert.utils -   guid: test-1
12/16/2025 16:38:48 - INFO - finbert.utils -   tokens: [CLS] the bristol port company has sealed a one million pound contract with cooper specialised handling to supply it with four 45 - ton ##ne , custom ##ised reach stack ##ers from ko ##ne ##cr ##ane ##s [SEP]
12/16/2025 16:38:48 - INFO - finbert.utils -   input_ids: 101 1996 7067 3417 2194 2038 10203 1037 2028 2454 9044 3206 2007 6201 17009 8304 2000 4425 2009 2007 2176 3429 1011 10228 2638 1010 7661 5084 3362 9991 2545 2013 12849 2638 26775 7231 2015 102 0 0 0 0 0 0 0 0 0 0
12/16/2025 16:38:48 - INFO - finbert.utils -   attention_mask: 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0
12/16/2025 16:38:48 - INFO - finbert.utils -   token_type_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
12/16/2025 16:38:48 - IN

Testing: 100%|██████████| 31/31 [00:00<00:00, 37.46it/s]


In [103]:
results['prediction'] = results.predictions.apply(lambda x: np.argmax(x,axis=0))

In [ ]:
def kd_report(df, cols=['label','prediction','logits']):
    #print('Validation loss:{0:.2f}'.format(metrics['best_validation_loss']))
    cs = CrossEntropyLoss(weight=finbert.class_weights)
    loss = cs(torch.tensor(list(df[cols[2]])),torch.tensor(list(df[cols[0]])))
    print("Loss:{0:.2f}".format(loss))
    print("Accuracy:{0:.2f}".format((df[cols[0]] == df[cols[1]]).sum() / df.shape[0]) )
    print("\nClassification Report:")
    return_val = classification_report(df[cols[0]], df[cols[1]], output_dict=True)
    
    
    new_report = {}
    
    for key in return_val.keys():
        new_report[key + "_post_kd"] = return_val[key]
    
    print(new_report)
    new_report["Loss_post_kd"] = loss
    new_report["Accuracy_post_kd"] = (df[cols[0]] == df[cols[1]]).sum() / df.shape[0]
    return new_report

In [ ]:
wandb_kd_report = kd_report(results,cols=['labels','prediction','predictions'])

summary = {
        "device": str(finbert.device),
        "model_dir": str(cl_path),
        "train_wall_s": float(train_wall_s),
        "train_examples": int(len(train_data)),
        "train_examples_per_s": float((len(train_data) * finbert.config.num_train_epochs) / train_wall_s)
        if train_wall_s > 0
        else float("inf"),
        "model_size_mb": float(get_model_size_mb(trained_model)),
        "profile_train_steps": finbert.config.profile_train_steps,
        **(finbert.profile_results.get("training_summary", {}) or {}),
        **eval_timing,
        **wandb_report,
        **wandb_kd_report,
    }

wandb.log(summary)
wandb.finish()

Loss:0.72
Accuracy:0.73

Classification Report:
{'0': {'precision': 0.6037037037037037, 'recall': 0.6104868913857678, 'f1-score': 0.6070763500931099, 'support': 267.0}, '1': {'precision': 0.5789473684210527, 'recall': 0.7734375, 'f1-score': 0.6622073578595318, 'support': 128.0}, '2': {'precision': 0.8506616257088847, 'recall': 0.782608695652174, 'f1-score': 0.8152173913043478, 'support': 575.0}, 'accuracy': 0.734020618556701, 'macro avg': {'precision': 0.6777708992778804, 'recall': 0.7221776956793139, 'f1-score': 0.6948336997523298, 'support': 970.0}, 'weighted avg': {'precision': 0.7468294709581363, 'recall': 0.734020618556701, 'f1-score': 0.7377339456503922, 'support': 970.0}}
